In [56]:
import pandas as pd
import altair as alt
import numpy as np

In [52]:
df_p = pd.read_pickle(f"./logs/current_and_future_predictions.pkl")
lines = pd.read_pickle(f"./logs/latest_line_all.pkl")
df_p = df_p.loc[df_p['FUTURE_GAME']==1]

In [53]:
stats = ["PTS", "REB", "AST", "FG3M", "BLK", "STL"]
stats_pred = [s+"_PREDICT" for s in stats]


In [54]:
non_stat_cols = ["PLAYER_NAME",
         "OPPONENT",
         "GAME_DATE"]

df_p = (df_p[non_stat_cols+['POSITION']+stats_pred]
        .groupby(non_stat_cols).agg("mean").reset_index()
        .merge(lines, left_on='PLAYER_NAME', right_on='PLAYER')
        .drop("PLAYER", axis=1)
       )

In [57]:
for stat in stats:
    #trying deltaF/F for "score" of deviation
    df_p[stat+"_dff"] = np.abs(df_p[stat+"_PREDICT"] - df_p[stat+"_LINE"]) / df_p[stat+"_LINE"]

In [62]:
to_keep = ["PLAYER_NAME", "OPPONENT", "GAME_DATE", "OPPONENT"]
suffixes = ["PREDICT", "LINE", "OVER", "UNDER", "dff"]

In [122]:
df_p_long = pd.wide_to_long(df_p, stubnames=stats, i = ["PLAYER_NAME","GAME_DATE"], j="MODEL", sep="_", suffix=r'\w+')[stats]

In [126]:
df_p_long.reset_index()

,PLAYER_NAME,GAME_DATE,MODEL,PTS,REB,AST,FG3M,BLK,STL
0,Al Horford,2022-06-08,PREDICT,10.789631,7.718660,2.155423,1.470188,0.878402,0.739143
1,Al Horford,2022-06-08,LINE,11.500000,8.500000,2.500000,1.500000,1.500000,0.500000
2,Al Horford,2022-06-08,OVER,-105.000000,-140.000000,-185.000000,-110.000000,135.000000,-170.000000
3,Al Horford,2022-06-08,UNDER,-120.000000,105.000000,135.000000,-115.000000,-185.000000,125.000000
4,Al Horford,2022-06-08,dff,0.061771,0.091922,0.137831,0.019875,0.414398,0.478287
...,...,...,...,...,...,...,...,...,...
65,Stephen Curry,2022-06-08,PREDICT,23.681373,4.550079,5.064559,3.697052,0.481706,1.243924
66,Stephen Curry,2022-06-08,LINE,28.500000,5.500000,5.500000,4.500000,0.500000,1.500000
67,Stephen Curry,2022-06-08,OVER,-110.000000,120.000000,110.000000,100.000000,240.000000,140.000000
68,Stephen Curry,2022-06-08,UNDER,-110.000000,-160.000000,-145.000000,-130.000000,-360.000000,-190.000000
